In [8]:
using Pkg
# Pkg.instantiate()
Pkg.status()

      Status `~/Mathematics/Conferences/Presentations/SumOfSquaresSlides/2022_ICM_Computational_Math/Project.toml`
  [7c1d4256] DynamicPolynomials v0.4.5
  [7073ff75] IJulia v1.23.3
  [4076af6c] JuMP v1.1.1
  [8bc5a954] PermutationGroups v0.3.2
  [c946c3f1] SCS v1.1.2
  [4b9e565b] SumOfSquares v0.6.2
  [24249f21] SymPy v1.1.6
  [858aa9a9] SymbolicWedderburn v0.3.0


In [9]:
using SymPy
using LinearAlgebra

# Example
> When is a quadratic polynomial in one variable decomposable as sum of squares?  

**Solution** Let us assume that $p = ax^2 + bx + c$ is an arbitrary polynomial in $\mathbb{R}[x]$. We may assume that $a > 0$.

In [10]:
a = symbols("a", positive=true)
x, b, c = symbols("x b c", real=true)

(x, b, c)

In [11]:
p = a*x^2 + b*x + c

   2          
a⋅x  + b⋅x + c

Let us define the monomial basis: $m_1 = [1, x]$ (as a row vector)

In [12]:
m₁ = transpose(Sym[1, x])

1×2 transpose(::Vector{Sym}) with eltype Sym:
 1  x

and define $P$ as

In [13]:
P = Sym[c b/2; b/2 a]

2×2 Matrix{Sym}:
   c  b/2
 b/2    a

Let us check, that the $m·P·m^T$ does what it is supposed to do:

In [14]:
ex = simplify(m₁*P*m₁')
@assert ex == p
ex

   2          
a⋅x  + b⋅x + c

Now we take $LDL^T$-decomposition of $P$, i.e. we find: a **lower-triangular** matrix $L$ and a **diagonal** matrix $D$ such that $$L·D·L^T = P$$

$$L·\sqrt D · (\sqrt D )^T·L^T = L·\sqrt D · \left(L·\sqrt D \right)^T = P$$

Then $\sqrt P  = L\cdot\sqrt D$

In [15]:
L, D = sympy.Matrix(P).LDLdecomposition()
@assert L*D*L' == P
L

2×2 Matrix{Sym}:
       1  0
 b/(2*c)  1

Using $L$ and $D$ we may define $$Q = L·\sqrt{D},$$ which is a matrix square root of $P$, i.e. 
$$Q·Q^T = \left(L\sqrt{D}\right)·\left(L\sqrt{D}\right)^T = L·\left(\sqrt{D}\sqrt{D}\right)·L^T = P.$$

Thus the columns of $Q$ provide candidates for summands in SOS-decomposition:
$$ p = [1,x]·P·\begin{bmatrix}1\\x\end{bmatrix}
= \left([1,x]·(L\sqrt{D})\right)·\left([1,x]·(L\sqrt{D})\right)^T = \left([1,x]Q\right)·\left([1,x]Q\right)^T
$$

In [16]:
Q = L*sqrt.(D)
Q

2×2 Matrix{Sym}:
       √c                    0
 b/(2*sqrt(c))  sqrt(a - b^2/(4*c))

remember we want to express $p$ as a sum of (two) squares: $f₁^2 + f₂^2$

In [17]:
f = m₁*Q

1×2 transpose(::Vector{Sym}) with eltype Sym:
 b*x/(2*sqrt(c)) + sqrt(c)  x*sqrt(a - b^2/(4*c))

In [18]:
simplify(f[1]^2 + f[2]^2) == p

true

For this to be a sum of squares we just need that $c>0$ and $$a - \frac{b^2}{4c} ≥ 0,$$
i.e.
$$b^2 - 4ac ≤ 0.$$

# Example
> Formulate the sum of squares decomposition problem as the feasibility of a semi-definite program for a polynomial in one variable of degree no larger than $4$.

**Solution**: Let $f\in \mathbb{R}[x]$ be an arbitrary polynomial in $x$ of degree not greater than $4$, i.e.
$$f = C_0 + C_1x + C_2x^2 + C_3x^3 + C_4x^4.$$

This time our monomial basis $m_2$ will consist of three elements:

In [19]:
m₂ = [1 x x^2]

1×3 Matrix{Sym}:
 1  x  x^2

In [20]:
P = let p = collect(symbols("p(0:3)(0:3)", real=true))
    P = Symmetric(reshape(p, 3,3))
end

3×3 Symmetric{Sym, Matrix{Sym}}:
 p₀₀  p₁₀  p₂₀
 p₁₀  p₁₁  p₂₁
 p₂₀  p₂₁  p₂₂

We need to evaluate

In [24]:
ex = (m₂*P*m₂')[1]
collect(expand(ex), x)

                       3        4    2              
p₀₀ + 2⋅p₁₀⋅x + 2⋅p₂₁⋅x  + p₂₂⋅x  + x ⋅(p₁₁ + 2⋅p₂₀)

In [25]:
collect(dot(m₂'*m₂, P), x)

                       3        4    2              
p₀₀ + 2⋅p₁₀⋅x + 2⋅p₂₁⋅x  + p₂₂⋅x  + x ⋅(p₁₁ + 2⋅p₂₀)

How does the resulting polynomial $m·P·m^T$ depends on $P$? 

Comparing this to the given (arbitrary) polynomial $f$:

In [31]:
cfs = symbols("C0:5", real=true)
f = sum(cfs[i]*x^(i-1) for i in 1:5)

                2       3       4
C₀ + C₁⋅x + C₂⋅x  + C₃⋅x  + C₄⋅x 

In [32]:
pex = ex.as_poly(x)
pf = f.as_poly(x)
for (a,b) in zip(pex.coeffs(), pf.coeffs())
    display(Eq(a, b))
end

p₂₂ = C₄

2⋅p₂₁ = C₃

p₁₁ + 2⋅p₂₀ = C₂

2⋅p₁₀ = C₁

p₀₀ = C₀

So the final semi-definite problem is as follows:
$$\begin{align*}
\quad & \text{feasibility}\\
\text{Subject to:}\quad & P = (p_{i,j}) \succcurlyeq 0\\
& p_{0,0} = C_0\\
& 2p_{0,1} = C_1\\
& 2p_{0,2} + p_{1,1} = C_2\\
& 2p_{1,2}= C_3\\
& p_{2,2} = C_4\\
\end{align*}$$

In [26]:
C = [1, 2, 3, 4, 5];
f = dot([1 x x^2 x^3 x^4], C)

   4      3      2          
5⋅x  + 4⋅x  + 3⋅x  + 2⋅x + 1

In [27]:
using JuMP
M = Model()
@variable M P[1:3,1:3] Symmetric
@constraints(M, begin # shifting indices by one!
    P[1,1] == C[1]
    2P[1,2] == C[2]
    2P[1,3] + P[1,1] == C[3]
    2P[2,3] == C[4]
    P[3,3] == C[5]
    P in PSDCone() # P >= 0
    end)
M

A JuMP Model
Feasibility problem with:
Variables: 6
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P

# Motzkin example

In [28]:
x,y = symbols("x y", real=true);
Motzkin = x^4*y^2 + x^2*y^4 - 3x^2*y^2 + 1

 4  2    2  4      2  2    
x ⋅y  + x ⋅y  - 3⋅x ⋅y  + 1

In [29]:
monomials(d::Int, x, y) = [x^(d-i)*y^(i) for i in 0:d]
function monomials(iter, x, y)
    m = Sym[]
    for d in iter
        append!(m, monomials(d, x, y))
    end
    return m
end
m₃ = transpose(monomials(0:3, x, y))

1×10 transpose(::Vector{Sym}) with eltype Sym:
 1  x  y  x^2  x⋅y  y^2  x^3  x^2*y  x*y^2  y^3

In [30]:
P = let l = length(m₃), p = collect(symbols("p(0:$l)(0:$l)", real=true))
    Symmetric(reshape(p, l,l))
end

10×10 Symmetric{Sym, Matrix{Sym}}:
 p₀₀  p₁₀  p₂₀  p₃₀  p₄₀  p₅₀  p₆₀  p₇₀  p₈₀  p₉₀
 p₁₀  p₁₁  p₂₁  p₃₁  p₄₁  p₅₁  p₆₁  p₇₁  p₈₁  p₉₁
 p₂₀  p₂₁  p₂₂  p₃₂  p₄₂  p₅₂  p₆₂  p₇₂  p₈₂  p₉₂
 p₃₀  p₃₁  p₃₂  p₃₃  p₄₃  p₅₃  p₆₃  p₇₃  p₈₃  p₉₃
 p₄₀  p₄₁  p₄₂  p₄₃  p₄₄  p₅₄  p₆₄  p₇₄  p₈₄  p₉₄
 p₅₀  p₅₁  p₅₂  p₅₃  p₅₄  p₅₅  p₆₅  p₇₅  p₈₅  p₉₅
 p₆₀  p₆₁  p₆₂  p₆₃  p₆₄  p₆₅  p₆₆  p₇₆  p₈₆  p₉₆
 p₇₀  p₇₁  p₇₂  p₇₃  p₇₄  p₇₅  p₇₆  p₇₇  p₈₇  p₉₇
 p₈₀  p₈₁  p₈₂  p₈₃  p₈₄  p₈₅  p₈₆  p₈₇  p₈₈  p₉₈
 p₉₀  p₉₁  p₉₂  p₉₃  p₉₄  p₉₅  p₉₆  p₉₇  p₉₈  p₉₉

In [31]:
sympy.MatMul(Matrix(m₃), P, Matrix(m₃'))

                                                                              
                                                                              
                                                                              
                                           ⎡p₀₀  p₁₀  p₂₀  p₃₀  p₄₀  p₅₀  p₆₀ 
                                           ⎢                                  
                                           ⎢p₁₀  p₁₁  p₂₁  p₃₁  p₄₁  p₅₁  p₆₁ 
                                           ⎢                                  
                                           ⎢p₂₀  p₂₁  p₂₂  p₃₂  p₄₂  p₅₂  p₆₂ 
                                           ⎢                                  
                                           ⎢p₃₀  p₃₁  p₃₂  p₃₃  p₄₃  p₅₃  p₆₃ 
                                           ⎢                                  
⎡          2        2   3   2       2   3⎤ ⎢p₄₀  p₄₁  p₄₂  p₄₃  p₄₄  p₅₄  p₆₄ 
⎣1  x  y  x   x⋅y  y   x   x ⋅y  x⋅y   y ⎦⋅⎢        

In [32]:
f = dot(m₃'*m₃, P)

                     2                              2          2          3   
p₀₀ + 2⋅p₁₀⋅x + p₁₁⋅x  + 2⋅p₂₀⋅y + 2⋅p₂₁⋅x⋅y + p₂₂⋅y  + 2⋅p₃₀⋅x  + 2⋅p₃₁⋅x  + 

       2          4                      2              2          3          
2⋅p₃₂⋅x ⋅y + p₃₃⋅x  + 2⋅p₄₀⋅x⋅y + 2⋅p₄₁⋅x ⋅y + 2⋅p₄₂⋅x⋅y  + 2⋅p₄₃⋅x ⋅y + p₄₄⋅x

2  2          2            2          3          2  2            3        4   
 ⋅y  + 2⋅p₅₀⋅y  + 2⋅p₅₁⋅x⋅y  + 2⋅p₅₂⋅y  + 2⋅p₅₃⋅x ⋅y  + 2⋅p₅₄⋅x⋅y  + p₅₅⋅y  + 

       3          4          3            5          4            3  2        
2⋅p₆₀⋅x  + 2⋅p₆₁⋅x  + 2⋅p₆₂⋅x ⋅y + 2⋅p₆₃⋅x  + 2⋅p₆₄⋅x ⋅y + 2⋅p₆₅⋅x ⋅y  + p₆₆⋅x

6          2            3            2  2          4            3  2          
  + 2⋅p₇₀⋅x ⋅y + 2⋅p₇₁⋅x ⋅y + 2⋅p₇₂⋅x ⋅y  + 2⋅p₇₃⋅x ⋅y + 2⋅p₇₄⋅x ⋅y  + 2⋅p₇₅⋅x

2  3          5          4  2            2          2  2            3         
 ⋅y  + 2⋅p₇₆⋅x ⋅y + p₇₇⋅x ⋅y  + 2⋅p₈₀⋅x⋅y  + 2⋅p₈₁⋅x ⋅y  + 2⋅p₈₂⋅x⋅y  + 2⋅p₈₃⋅

 3  2          2  3            4          4  2

In [33]:
pf = sympy.Poly(f, x,y)
pm = sympy.Poly(Motzkin, x,y)

Poly(x**4*y**2 + x**2*y**4 - 3*x**2*y**2 + 1, x, y, domain='ZZ')

In [34]:
for m in monomials(0:6, x, y)
    display(Eq(pf.coeff_monomial(m), pm.coeff_monomial(m)))
end

p₀₀ = 1

2⋅p₁₀ = 0

2⋅p₂₀ = 0

p₁₁ + 2⋅p₃₀ = 0

2⋅p₂₁ + 2⋅p₄₀ = 0

...

p₇₇ + 2⋅p₈₆ = 1

2⋅p₈₇ + 2⋅p₉₆ = 0

p₈₈ + 2⋅p₉₇ = 1

2⋅p₉₈ = 0

p₉₉ = 0

# Proving that Motzkin is not a SOS

**Solution**: Suppose that Motzkin polynomial **IS a sum of squares** of polynomials and take one summand $f^2$ of the sum.

In [36]:
Motzkin

 4  2    2  4      2  2    
x ⋅y  + x ⋅y  - 3⋅x ⋅y  + 1

In [37]:
a₀ = symbols("a0", real=true)
a = collect(symbols("a1:10", real=true))
f = sum(c*m for (c,m) in zip([a₀;a], m₃))

                       2                2       3       2           2       3
a₀ + a₁⋅x + a₂⋅y + a₃⋅x  + a₄⋅x⋅y + a₅⋅y  + a₆⋅x  + a₇⋅x ⋅y + a₈⋅x⋅y  + a₉⋅y 

In [38]:
sympy.Poly(expand(f^2), x, y)(
#     a[6] => 0, # x⁶
#     a[9] => 0, # y⁶
#     a[3] => 0, # x⁴
#     a[5] => 0, # y⁴
#     a[1] => 0, # x²
#     a[2] => 0, # y²
)

Poly(a6**2*x**6 + 2*a6*a7*x**5*y + 2*a3*a6*x**5 + (2*a6*a8 + a7**2)*x**4*y**2 
+ (2*a3*a7 + 2*a4*a6)*x**4*y + (2*a1*a6 + a3**2)*x**4 + (2*a6*a9 + 2*a7*a8)*x*
*3*y**3 + (2*a3*a8 + 2*a4*a7 + 2*a5*a6)*x**3*y**2 + (2*a1*a7 + 2*a2*a6 + 2*a3*
a4)*x**3*y + (2*a0*a6 + 2*a1*a3)*x**3 + (2*a7*a9 + a8**2)*x**2*y**4 + (2*a3*a9
 + 2*a4*a8 + 2*a5*a7)*x**2*y**3 + (2*a1*a8 + 2*a2*a7 + 2*a3*a5 + a4**2)*x**2*y
**2 + (2*a0*a7 + 2*a1*a4 + 2*a2*a3)*x**2*y + (2*a0*a3 + a1**2)*x**2 + 2*a8*a9*
x*y**5 + (2*a4*a9 + 2*a5*a8)*x*y**4 + (2*a1*a9 + 2*a2*a8 + 2*a4*a5)*x*y**3 + (
2*a0*a8 + 2*a1*a5 + 2*a2*a4)*x*y**2 + (2*a0*a4 + 2*a1*a2)*x*y + 2*a0*a1*x + a9
**2*y**6 + 2*a5*a9*y**5 + (2*a2*a9 + a5**2)*y**4 + (2*a0*a9 + 2*a2*a5)*y**3 + 
(2*a0*a5 + a2**2)*y**2 + 2*a0*a2*y + a0**2, x, y, domain='ZZ[a0,a1,a2,a3,a4,a5
,a6,a7,a8,a9]')

So if Motzikn polynomial $m(x,y)$ were a sum of squares,
$$
m(x,y) = \sum_i f_i^2(x,y)\enspace,
$$
its coefficient in front of $x^2y^2$ is the sum of the corresponding coefficients in $f_i$:
$$
-3 = \sum_i (f_i^2)_{x^2y^2} = \sum_i \left((a_4)_{(i)}\right)^2\enspace .
$$